# ChatGoodfire

这将帮助您开始使用 Goodfire [聊天模型](/docs/concepts/chat_models)。有关 ChatGoodfire 功能和配置的详细文档，请访问 [PyPI 项目页面](https://pypi.org/project/langchain-goodfire/)，或直接访问 [Goodfire SDK 文档](https://docs.goodfire.ai/sdk-reference/example)。所有 Goodfire 特有的功能（例如 SAE 功能、变体等）都可以通过主 `goodfire` 包获得。此集成是 Goodfire SDK 的包装器。

## 概述
### 集成详情

| 类 | 包 | 本地 | 可序列化 | JS 支持 | 包下载 | 包最新 |
| :--- | :--- | :---: | :---: |  :---: | :---: | :---: |
| [ChatGoodfire](https://python.langchain.com/api_reference/goodfire/chat_models/langchain_goodfire.chat_models.ChatGoodfire.html) | [langchain-goodfire](https://python.langchain.com/api_reference/goodfire/) | ❌ | ❌ | ❌ | ![PyPI - Downloads](https://img.shields.io/pypi/dm/langchain-goodfire?style=flat-square&label=%20) | ![PyPI - Version](https://img.shields.io/pypi/v/langchain-goodfire?style=flat-square&label=%20) |

### 模型功能
| [工具调用](/docs/how_to/tool_calling) | [结构化输出](/docs/how_to/structured_output/) | JSON 模式 | [图像输入](/docs/how_to/multimodal_inputs/) | 音频输入 | 视频输入 | [Token 级别流式输出](/docs/how_to/chat_streaming/) | 原生异步 | [Token 用量](/docs/how_to/chat_token_usage_tracking/) | [Logprobs](/docs/how_to/logprobs/) |
| :---: | :---: | :---: | :---: |  :---: | :---: | :---: | :---: | :---: | :---: |
| ❌ | ❌ | ❌ | ❌ | ❌ | ❌ | ✅ | ✅ | ✅ | ❌ |

## 设置

要访问 Goodfire 模型，您需要创建一个 Goodfire 账户，获取 API 密钥，并安装 `langchain-goodfire` 集成包。

### 凭据

前往 [Goodfire 设置](https://platform.goodfire.ai/organization/settings/api-keys) 注册 Goodfire 并生成 API 密钥。完成此操作后，设置 GOODFIRE_API_KEY 环境变量。

In [1]:
import getpass
import os

if not os.getenv("GOODFIRE_API_KEY"):
    os.environ["GOODFIRE_API_KEY"] = getpass.getpass("Enter your Goodfire API key: ")

为了启用对模型调用的自动跟踪，请设置您的 [LangSmith](https://docs.smith.langchain.com/) API 密钥：

In [ ]:
# os.environ["LANGSMITH_TRACING"] = "true"
# os.environ["LANGSMITH_API_KEY"] = getpass.getpass("Enter your LangSmith API key: ")

### 安装

LangChain 的 Goodfire 集成位于 `langchain-goodfire` 包中：

In [2]:
%pip install -qU langchain-goodfire

Note: you may need to restart the kernel to use updated packages.


## 实例化

现在我们可以实例化我们的模型对象并生成聊天补全了：

In [3]:
import goodfire
from langchain_goodfire import ChatGoodfire

base_variant = goodfire.Variant("meta-llama/Llama-3.3-70B-Instruct")

llm = ChatGoodfire(
    model=base_variant,
    temperature=0,
    max_completion_tokens=1000,
    seed=42,
)

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


## 调用

In [4]:
messages = [
    (
        "system",
        "You are a helpful assistant that translates English to French. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]
ai_msg = await llm.ainvoke(messages)
ai_msg

AIMessage(content="J'adore la programmation.", additional_kwargs={}, response_metadata={}, id='run-8d43cf35-bce8-4827-8935-c64f8fb78cd0-0', usage_metadata={'input_tokens': 51, 'output_tokens': 39, 'total_tokens': 90})

In [5]:
print(ai_msg.content)

J'adore la programmation.


## 链式调用

我们可以像这样将我们的模型与提示模板进行[链式调用](/docs/how_to/sequence/)：

In [6]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate(
    [
        (
            "system",
            "You are a helpful assistant that translates {input_language} to {output_language}.",
        ),
        ("human", "{input}"),
    ]
)

chain = prompt | llm
await chain.ainvoke(
    {
        "input_language": "English",
        "output_language": "German",
        "input": "I love programming.",
    }
)

AIMessage(content='Ich liebe das Programmieren. How can I help you with programming today?', additional_kwargs={}, response_metadata={}, id='run-03d1a585-8234-46f1-a8df-bf9143fe3309-0', usage_metadata={'input_tokens': 46, 'output_tokens': 46, 'total_tokens': 92})

## Goodfire 特有功能

要使用 Goodfire 特有的功能，例如 SAE 功能和变体，您可以直接使用 `goodfire` 程序包。

In [7]:
client = goodfire.Client(api_key=os.environ["GOODFIRE_API_KEY"])

pirate_features = client.features.search(
    "assistant should roleplay as a pirate", base_variant
)
pirate_features

FeatureGroup([
   0: "The assistant should adopt the persona of a pirate",
   1: "The assistant should roleplay as a pirate",
   2: "The assistant should engage with pirate-themed content or roleplay as a pirate",
   3: "The assistant should roleplay as a character",
   4: "The assistant should roleplay as a specific character",
   5: "The assistant should roleplay as a game character or NPC",
   6: "The assistant should roleplay as a human character",
   7: "Requests for the assistant to roleplay or pretend to be something else",
   8: "Requests for the assistant to roleplay or pretend to be something",
   9: "The assistant is being assigned a role or persona to roleplay"
])

In [8]:
pirate_variant = goodfire.Variant("meta-llama/Llama-3.3-70B-Instruct")

pirate_variant.set(pirate_features[0], 0.4)
pirate_variant.set(pirate_features[1], 0.3)

await llm.ainvoke("Tell me a joke", model=pirate_variant)

AIMessage(content='Why did the scarecrow win an award? Because he was outstanding in his field! Arrr! Hope that made ye laugh, matey!', additional_kwargs={}, response_metadata={}, id='run-7d8bd30f-7f80-41cb-bdb6-25c29c22a7ce-0', usage_metadata={'input_tokens': 35, 'output_tokens': 60, 'total_tokens': 95})

## API 参考

有关 ChatGoodfire 所有功能和配置的详细文档，请访问 [API 参考](https://python.langchain.com/api_reference/goodfire/chat_models/langchain_goodfire.chat_models.ChatGoodfire.html)。